# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [2]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.6 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [4]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [6]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [8]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [11]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE song (
      id INTEGER,
      name STRING,
      interpreter STRING,
      creation INTEGER
    );

    CREATE TABLE band (
      id INTEGER,
      name STRING,
      number_players INTEGER,
      years_active INTEGER,
      genre STRING
    );

    CREATE TABLE album (
      id INTEGER,
      name STRING,
      number_songs INTEGER,
      year INTEGER,
      id_band INTEGER,
      id_song INTEGER
    );

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [12]:
sp_nl2sql = sp_nl2sql.format(question="What are the names of all songs created after 2000?")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE song (
      id INTEGER,
      name STRING,
      interpreter STRING,
      creation INTEGER
    );

    CREATE TABLE band (
      id INTEGER,
      name STRING,
      number_players INTEGER,
      years_active INTEGER,
      genre STRING
    );

    CREATE TABLE album (
      id INTEGER,
      name STRING,
      number_songs INTEGER,
      year INTEGER,
      id_band INTEGER,
      id_song INTEGER
    );

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `What are the names of all songs created after 2000?`:
    ```sq

In [13]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [14]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [15]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT s.name FROM song s JOIN album a ON s.id = a.id_song JOIN band b ON a.id_band = b.id WHERE s.creation > to_date('2000-01-01', 'YYYY-MM-DD') ORDER BY s.name NULLS LAST;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [16]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   CREATE TABLE song (
      id INTEGER,
      name STRING,
      interpreter STRING,
      creation INTEGER
    );

    CREATE TABLE band (
      id INTEGER,
      name STRING,
      number_players INTEGER,
      years_active INTEGER,
      genre STRING
    );

    CREATE TABLE album (
      id INTEGER,
      name STRING,
      number_songs INTEGER,
      year INTEGER,
      id_band INTEGER,
      id_song INTEGER
    );

    ### Examples:
    ### Question
    List all bands with more than 4 members.

    ### Response:
    ### SQL
    ```sql
    SELECT * FROM band
    WHERE number_players > 4

    ### Question
    Get all albums released in 1999.

    ### Response:
    ### SQL
    ```sql
    SELECT * FROM album
    WHERE year = 1999

    ### Question
    Get all bands that have an album released after 2000.

    ### Response:
    ### SQL
    ```sql
    SELECT DISTINCT b.*
    FROM band b
    JOIN album a ON b.id = a.id_band
    WHERE a.year > 2000;

    `{question}`:
    ```sql3
    """


In [17]:
sp_nl2sql2 = sp_nl2sql2.format(question="What are the names of all songs created after 2000")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   CREATE TABLE song (
      id INTEGER,
      name STRING,
      interpreter STRING,
      creation INTEGER
    );

    CREATE TABLE band (
      id INTEGER,
      name STRING,
      number_players INTEGER,
      years_active INTEGER,
      genre STRING
    );

    CREATE TABLE album (
      id INTEGER,
      name STRING,
      number_songs INTEGER,
      year INTEGER,
      id_band INTEGER,
      id_song INTEGER
    );

    ### Examples: 
    ### Question
    List all bands with more than 4 members.

  

In [18]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [19]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT s.name
    FROM song s
    JOIN album a ON s.id = a.id_song
    JOIN band b ON a.id_band = b.id
    WHERE a.year > 2000
    ORDER BY s.name NULLS LAST;


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [20]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

       CREATE TABLE song (
      id INTEGER,
      name STRING,
      interpreter STRING,
      creation INTEGER
    );

    CREATE TABLE band (
      id INTEGER,
      name STRING,
      number_players INTEGER,
      years_active INTEGER,
      genre STRING
    );

    CREATE TABLE album (
      id INTEGER,
      name STRING,
      number_songs INTEGER,
      year INTEGER,
      id_band INTEGER,
      id_song INTEGER
    );

    ### Samples

    ### Question
    List all bands with more than 4 members.

    ### SQL
    ```sql
    SELECT * FROM band
    WHERE number_players > 4

    ### Question
    Get all albums released in 1999.

    ### SQL
    ```sql
    SELECT * FROM album
    WHERE year = 1999

    ### Question
    Get all bands that have an album released after 2000.

    ### SQL
    ```sql
    SELECT DISTINCT b.*
    FROM band b
    JOIN album a ON b.id = a.id_band
    WHERE a.year > 2000;

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [21]:
sp_nl2sql3 = sp_nl2sql3b.format(question="What are the names of all songs created after 2000")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

       CREATE TABLE song (
      id INTEGER,
      name STRING,
      interpreter STRING,
      creation INTEGER
    );

    CREATE TABLE band (
      id INTEGER,
      name STRING,
      number_players INTEGER,
      years_active INTEGER,
      genre STRING
    );

    CREATE TABLE album (
      id INTEGER,
      name STRING,
      number_songs INTEGER,
      year INTEGER,
      id_band INTEGER,
      id_song INTEGER
    );
    
    ### Samples
    
    ### Question
    List all bands with more than 4 me

In [22]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [23]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT s.name, to_char(s.creation, 'YYYY-MM-DD') AS creation_date
    FROM song s
    WHERE to_date(to_char(s.creation, 'YYYY-MM-DD'), 'YYYY-MM-DD') > to_date('2000-01-01', 'YYYY-MM-DD');


#Now the question in spanish.


In [25]:
sp_nl2sql3 = sp_nl2sql3b.format(question="todas las canciones creadas despues del 2000")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

       CREATE TABLE song (
      id INTEGER,
      name STRING,
      interpreter STRING,
      creation INTEGER
    );

    CREATE TABLE band (
      id INTEGER,
      name STRING,
      number_players INTEGER,
      years_active INTEGER,
      genre STRING
    );

    CREATE TABLE album (
      id INTEGER,
      name STRING,
      number_songs INTEGER,
      year INTEGER,
      id_band INTEGER,
      id_song INTEGER
    );
    
    ### Samples
    
    ### Question
    List all bands with more than 4 me

In [26]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [27]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT DISTINCT b.*
    FROM band b
    JOIN album a ON b.id = a.id_band
    WHERE a.year > 2000;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

1. Prompt without shots: This prompt produced a valid query, but it included unnecessary joins with other tables. It also added things like NULLS LAST and date formatting that weren’t required by the question. The formatting structure was not correct although it generally got to what was intended.

2. OpenAI Style Few-Shot Prompt: This version changed the filtering condition to album.year > 2000 instead of using song.creation. The query still worked, but the logic shifted a bit. probably due to ambiguity in the schema or prompt wording. Overall, the SQL output was clean and structurally sound.

3. Sample-Style Prompt: In this case, the model focused only on the song table, filtering by song.creation, but added unnecessary to_char/to_date conversions. While technically correct, these transformations just made the query more complex without adding real value.

4. Spanish Prompt: When I asked the same question in Spanish, the model gave a completely different answer. Instead of listing songs, it returned bands with albums released after 2000. This seemed to be a clear example of the model misinterpreting the question, probably due to translation limitations or semantic confusion.


Were there variations that didn't work well?
The version were I asked the question in Spanish did not work well. Instead of returning a list of songs created after 2000, the query returned all bands that had albums released after 2000. This was a clear misunderstanding of the prompt. It seems that when switching to Spanish, the model may interpret things more loosely or default to safer patterns based on common examples in its training.